<a href="https://colab.research.google.com/github/wolfinwallst/Machine_Learning_Deep_basis/blob/main/RL_Actor_Critic_CartPole_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[블로그 글 참조](https://baechu-story.tistory.com/57) 하고, 잘 작동하게 수정했다.

추가로 이해 안 되는 부분 설명도 추가함

여기서 구현한 방법인 `TD Actor-Critic` 이다:

In [20]:
import gym

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.distributions import Categorical
# Categorical은 discrete 확률분포를 다룰 때 사용

In [21]:
# hyperparameters
learning_rate = 0.0002
gamma         = 0.98
n_rollout     = 10 # 몇 번의 step 마다 데이터를 업데이트 할지 (업데이트 주기)

In [22]:
class ActorCritic(nn.Module):
    def __init__(self):
        super(ActorCritic, self).__init__()
        self.data = [] # 경험 데이터를 저장할 리스트

        self.fc1 = nn.Linear(4,256) # 상태(4차원)를 256차원으로 매핑하는 FC layer
        self.fc_pi = nn.Linear(256,2) #  policy 네트웤: 256차원 -> 행동 개수(2)로 매핑하여 행동 확률을 계산
        self.fc_v = nn.Linear(256,1) # value 네트웤: 256차원 -> 상태의 가치를 출력 (스칼라)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    # actor 네트웤, 순서: input-fc1-relu-fc_pi-softmax
    def pi(self, x, softmax_dim = 0): # 정책 함수: 주어진 상태에서 행동 확률 분포를 반환
        x = F.relu(self.fc1(x))
        x = self.fc_pi(x) # (policy) logit 계산
        prob = F.softmax(x, dim=softmax_dim)
        return prob

    # critic 네트웤, 순서: input-fc1-relu-fc_v
    def v(self, x): # 가치 함수: 주어진 상태의 가치를 반환
        x = F.relu(self.fc1(x))
        v = self.fc_v(x)
        return v

    def put_data(self, transition): # 하나의 transition 데이터를 저장하는 함수
        self.data.append(transition)

    def make_batch(self): # 저장된 경험 데이터를 배치 텐서로 변환하는 함수
        s_lst, a_lst, r_lst, s_next_lst, done_lst = [], [], [], [], []
        for transition in self.data:
            s, a, r, s_next, done = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r / 100.0]) # 보상 값을 스케일 조정 (옵션)
            s_next_lst.append(s_next)
            done_mask = 0.0 if done else 1.0 # 에피소드 종료 여부에 따른 마스킹
            done_lst.append([done_mask])

        # 리스트를 텐서로 변환
        s_batch = torch.tensor(s_lst, dtype=torch.float)
        a_batch = torch.tensor(a_lst)
        r_batch = torch.tensor(r_lst, dtype=torch.float)
        s_next_batch = torch.tensor(s_next_lst, dtype=torch.float)
        done_batch = torch.tensor(done_lst, dtype=torch.float)
        self.data = []
        return s_batch, a_batch, r_batch, s_next_batch, done_batch

    def train_net(self):
        s, a, r, s_next, done = self.make_batch()
        # TD Target 계산 : r + γ * V(s')
        td_target = r + gamma * self.v(s_next) * done
        # TD 에러 (Advantage) 계산
        delta = td_target - self.v(s)

        pi = self.pi(s, softmax_dim=1) # 정책 네트워크로부터 각 액션의 확률 pi 계산 (배치 형태)
        pi_a = pi.gather(1, a) # 각 상태에서 실제 선택된 행동 a에 해당하는 확률만 추출 (즉 batch에서 뽑은 action의 확률)

        # 정책 함수와 가치 함수의 loss를 합산하여 총 손실 계산
        # 현재 상태의 가치를 최대화하여 다음 상태의 가치와 차이를 줄이는 방향으로 학습을 진행
        loss = -torch.log(pi_a) * delta.detach() + F.smooth_l1_loss(self.v(s), td_target.detach())

        self.optimizer.zero_grad()
        loss.mean().backward()
        self.optimizer.step()

`pi = self.pi(s, softmax_dim=1)` 에서 softmax_dim=1로 지정하면, 각 행(각 상태)에 대해 확률 분포가 계산된다. 예를 들어, 상태 배치가 (batch_size, num_features) 형태이고, 마지막 출력이 (batch_size, num_actions)라면, 각 row에 대해 softmax를 적용해 각 행동이 선택될 확률을 얻는다.

`gather(dim, index)` 함수는 지정된 차원 dim을 따라, index 텐서에 있는 인덱스에 해당하는 값을 모은다.
예를 들어, pi가 (batch_size, num_actions) 텐서이고, a가 (batch_size, 1) 텐서라면,
pi.gather(1, a)는 각 행(row)마다 a에 기록된 인덱스 위치의 값을 추출하여 (batch_size, 1) 형태의 텐서를 반환한다.

`torch.log(pi_a)` 는 수식으로
$$ \log \pi(a | s) $$
이다.

위 코드의 `loss`는 policy loss와 value loss의 합으로

$$ L = L_{\text{policy}} + L_{\text{value}} = \log \pi(a | s) \dot \delta + \text{Smooth L1 loss} (V(s), r + \gamma V(s'))$$

를 나타낸다. 이 손실을 최소화하도록 네트워크 파라미터를 업데이트하는 것이 코드의 목적이다.

`detach()` 함수는 해당 텐서를 계산 그래프에서 분리시켜, 이 값들이 역전파 시 그래디언트 계산에 영향을 주지 않도록 한다. 이는 정책 손실과 가치 손실을 계산할 때 advantage(𝛿)와 TD target이 상수처럼 취급되게 하여,
네트워크 파라미터 업데이트 시 부정확한 그라디언트 전파를 막아준다.

delta를 상수 값으로 두기 위해 detach() 사용,
마찬가지로 `td_target`을 `detach`하여, 이 값이 역전파 시 가치 네트워크 외의 다른 부분(예: 정책 네트워크)에 영향을 미치지 않도록 한다.

In [23]:
def main():
    env = gym.make('CartPole-v1')
    model = ActorCritic()
    print_interval = 20 # 성능 출력 간격
    score = 0.0

    for n_epi in range(1000): # 1000 번으로 변경
        done = False
        s = env.reset()

        while not done:
            # n_rollout 번 만큼 환경과 상호작용 후 학습 진행
            for t in range(n_rollout):
                prob = model.pi(torch.from_numpy(s).float())
                m = Categorical(prob)
                a = m.sample().item()
                s_next, r, done, info = env.step(a)
                model.put_data((s, a, r, s_next, done))

                s = s_next
                score += r

                if done:
                    break

            model.train_net() # 저장된 경험 데이터를 이용해 네트웤 업데이트

        if n_epi % print_interval==0 and n_epi!=0:
            print("# of episode :{}, avg score : {:.1f}".format(n_epi, score/print_interval))
            score = 0.0
    env.close()

if __name__ == '__main__':
    main()

/usr/local/lib/python3.11/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


# of episode :20, avg score : 24.6
# of episode :40, avg score : 22.5
# of episode :60, avg score : 23.6
# of episode :80, avg score : 20.4
# of episode :100, avg score : 23.6
# of episode :120, avg score : 25.9
# of episode :140, avg score : 24.8
# of episode :160, avg score : 23.8
# of episode :180, avg score : 28.3
# of episode :200, avg score : 29.3
# of episode :220, avg score : 31.1
# of episode :240, avg score : 24.4
# of episode :260, avg score : 33.9
# of episode :280, avg score : 37.0
# of episode :300, avg score : 41.0
# of episode :320, avg score : 37.8
# of episode :340, avg score : 43.0
# of episode :360, avg score : 44.1
# of episode :380, avg score : 52.2
# of episode :400, avg score : 64.7
# of episode :420, avg score : 54.8
# of episode :440, avg score : 68.3
# of episode :460, avg score : 52.3
# of episode :480, avg score : 60.1
# of episode :500, avg score : 84.5
# of episode :520, avg score : 97.3
# of episode :540, avg score : 116.3
# of episode :560, avg score : 

 코드를 직접 실행하면 `__name__`의 값이 "`__main__`"이 되어 main() 함수가 호출되지만, 다른 모듈에서 import할 때는 자동으로 실행되지 않는다. 이는 파이썬에서 모듈을 깔끔하게 설계하고 재사용할 수 있도록 하는 중요한 패턴이다.